In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from tool_evaluator_agent.tools import ScrapingFishTool

In [20]:
import os
SCRAPING_FISH_API_KEY = os.getenv("SCRAPING_FISH_API_KEY")


In [3]:
content = ScrapingFishTool().run('https://www.orulo.com.br')

Using Tool: Scrape website content


In [4]:
print(content)


Scrapped Content: A maior plataforma de imóveis novos do Brasil

Somos o marketplace de imóveis residenciais e comerciais mais completo do país, otimizando a geração de oportunidades de negócios de agentes imobiliários e incorporadoras.

Integramos o ecossistema de parcerias imobiliárias

Criamos soluções que conectam corretores, imobiliárias e incorporadoras, eliminando atritos nas transações imobiliárias.

A maior evolução que o mercado de imóveis já viu

A primeira plataforma B2B de distribuição de unidades de imóveis integrada a diversos sistemas de CRMs imobiliários.

A plataforma que empodera corretores de imóveis

Simplificamos a jornada de atendimento, acelerando as vendas. Baixe o App gratuitamente e atenda clientes com facilidade em qualquer lugar.

Integração e distribuição

Incorporadoras integram seus lançamentos em qualquer operação imobiliária, além de distribuir suas unidades via interfaces mobile e desktop Órulo.

O mercado imobiliário onde quer que você esteja

Utili

In [2]:
import os
from google.oauth2 import service_account
from langchain_google_genai import ChatGoogleGenerativeAI

# Set the path to the service account JSON key file
service_account_file = os.getenv("GOOGLE_SERVICE_ACCOUNT_FILE")

# Authenticate using the service account key
credentials = service_account.Credentials.from_service_account_file(service_account_file)


/Users/aolchik/mambaforge/envs/crewai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash',
                            verbose=True,
                            temperature=0.5,
                            credentials=credentials)

I0000 00:00:1724249811.931081 19971561 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1724249811.931865 19971561 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [7]:
print(type(llm))

<class 'langchain_google_genai.chat_models.ChatGoogleGenerativeAI'>


In [8]:
isinstance(llm, ChatGoogleGenerativeAI)

True

In [13]:
llm.model.split('/')[-1]

'gemini-1.5-flash'

In [4]:
llm.invoke("Sing a ballad of LangChain.")

AIMessage(content="(Verse 1)\nIn the realm of code, where data flows,\nA tool was born, LangChain, it goes.\nWith chains of thought, it weaves its spell,\nTo unlock knowledge, secrets to tell.\n\n(Chorus)\nOh, LangChain, LangChain, a symphony of code,\nConnecting minds, a path we've strode.\nWith prompts and chains, we tap the well,\nOf language models, stories to tell.\n\n(Verse 2)\nFrom LLM's power, we draw our might,\nTo answer questions, day and night.\nWith prompts we guide, with chains we bind,\nA tapestry of words, for all to find.\n\n(Chorus)\nOh, LangChain, LangChain, a symphony of code,\nConnecting minds, a path we've strode.\nWith prompts and chains, we tap the well,\nOf language models, stories to tell.\n\n(Verse 3)\nThrough agents and tools, it takes its stand,\nTo automate tasks, with a helping hand.\nFrom summarization to text creation,\nLangChain empowers, with innovation.\n\n(Chorus)\nOh, LangChain, LangChain, a symphony of code,\nConnecting minds, a path we've strode.

In [14]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o',
                  verbose=True,
                  temperature=0)

In [15]:
llm.model_name

'gpt-4o'

In [17]:
from langchain_community.llms import Ollama
llm = Ollama(model='openhermes')

In [18]:
isinstance(llm, Ollama)

True

In [19]:
llm.model

'openhermes'